In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string

In [3]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        detect_short = range_width * self.K < ohlcv_latest["high"][0] - ohlcv_latest["open"][0]
        detect_long  = range_width * self.K < ohlcv_latest["open"][0] - ohlcv_latest["low"][0]

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.make_order("Buy", session.buy_board_edge_price, 0.01, 600, "Open Long")    
                else:

                    session.make_order("Buy", session.buy_board_edge_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.make_order("Sell", session.sell_board_edge_price, 0.01, 600, "Open Short") 
                else:
                    session.make_order("Sell", session.sell_board_edge_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [4]:
bn = BinanceMarket("BTCBUSD")
bn.download(10)

rbot.init_debug_log()

back_tester = BackTester("BN", "BTCBUSD", True)

print("start")
r = back_tester.run(Agent())
print("end")

print(r)



start
2022-12-06T14:31:12.930Z DEBUG [rbot::sim::back] want on tick  false
2022-12-06T14:31:12.930Z DEBUG [rbot::sim::back] want on clock true
2022-12-06T14:31:12.930Z DEBUG [rbot::sim::back] want on event true
2022-12-06T14:31:12.930Z DEBUG [rbot::sim::back] clock interval 7200
2022-12-06T14:31:12.930Z DEBUG [rbot::db::sqlite] Database open path = /Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/BTCBUSD.db
2022-12-06T14:31:12.931Z DEBUG [rbot::db::sqlite] Database open path = /Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/BTCBUSD.db
2022-12-06T14:31:12.931Z DEBUG [rbot::db::sqlite] cache update all 2022-10-04T12:00:00.000000 -> 2022-10-05T00:00:05.260000
2022-12-06T14:31:12.931Z DEBUG [rbot::db::sqlite] create iter 0 microsec
2022-12-06T14:31:12.931Z DEBUG [rbot::db::df] ohlcv_df, from=2022-10-04T12:00:00.000000 / to=2022-10-05T00:00:05.260000
2022-12-06T14:31:12.931Z DEBUG [rbot::db::df] Select from 2022-10-04T12:00:00.000000 -> 2022-10-0

In [5]:
import rbot
df = rbot.result_to_df(r)

In [6]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_home_size,open_foreign_size,close_price,close_home_size,close_foreign_size,order_price,order_home_size,order_foreign_size,profit,fee,total_profit,message,sum_profit
create_time,,,,,,,,,,,,,,,,,,,,
2022-10-05 17:59:59.896000+00:00,2022-10-05 18:00:00.653000+00:00,0000-0001,0,Sell,True,Open,20168.24,0.01,4.958291e-07,0.00,0.00,0.000000e+00,20168.24,0.01,4.958291e-07,0.000000e+00,4.958291e-11,-4.958291e-11,Open Short,-4.958291e-11
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,0,Buy,True,Close,20168.24,0.01,4.958291e-07,19605.10,0.01,5.100714e-07,19605.10,0.01,5.100714e-07,-1.424227e-08,5.100714e-11,-1.429328e-08,doten Long,-1.434286e-08
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,1,Buy,True,Open,19605.10,0.01,5.100714e-07,0.00,0.00,0.000000e+00,19605.10,0.01,5.100714e-07,0.000000e+00,5.100714e-11,-5.100714e-11,doten Long,-1.439387e-08
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,0,Sell,True,Close,19605.10,0.01,5.100714e-07,19474.00,0.01,5.135052e-07,19474.00,0.01,5.135052e-07,3.433827e-09,5.135052e-11,3.382477e-09,Doten Short,-1.101139e-08
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,1,Sell,True,Open,19474.00,0.01,5.135052e-07,0.00,0.00,0.000000e+00,19474.00,0.01,5.135052e-07,0.000000e+00,5.135052e-11,-5.135052e-11,Doten Short,-1.106274e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-03 21:59:59.022000+00:00,2022-12-03 22:00:02.010000+00:00,0000-0050,1,Buy,True,Open,16949.21,0.01,5.899980e-07,0.00,0.00,0.000000e+00,16949.21,0.01,5.899980e-07,0.000000e+00,5.899980e-11,-5.899980e-11,doten Long,1.150719e-07
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,0,Sell,True,Close,16949.21,0.01,5.899980e-07,16968.66,0.01,5.893217e-07,16968.66,0.01,5.893217e-07,-6.762739e-10,5.893217e-11,-7.352060e-10,Doten Short,1.143367e-07
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,1,Sell,True,Open,16968.66,0.01,5.893217e-07,0.00,0.00,0.000000e+00,16968.66,0.01,5.893217e-07,0.000000e+00,5.893217e-11,-5.893217e-11,Doten Short,1.142777e-07


In [ ]:
df['total_profit'].sum()

In [5]:
r


[update_time: 1664992800653000, order_id: "0000-0001", order_sub_id: 0, order_side: Sell, post_only: true, create_time: 1664992799896000, status: OpenPosition, open_price: 20168.24, open_home_size: 0.01, open_foreign_size: 4.958290857308322e-7, close_price: 0.0, close_home_size: 0.0, close_foreign_size: 0.0, order_price: 20168.24, order_home_size: 0.01, order_foreign_size: 4.958290857308322e-7, profit: 0.0, fee: 4.9582908573083223e-11, total_profit: -4.9582908573083223e-11, message: "Open Short",
 update_time: 1665151201114000, order_id: "0000-0002", order_sub_id: 0, order_side: Buy, post_only: true, create_time: 1665151199995000, status: ClosePosition, open_price: 20168.24, open_home_size: 0.01, open_foreign_size: 4.958290857308322e-7, close_price: 19605.1, close_home_size: 0.01, close_foreign_size: 5.100713589831218e-7, order_price: 19605.1, order_home_size: 0.01, order_foreign_size: 5.100713589831218e-7, profit: -1.4242273252289545e-8, fee: 5.100713589831218e-11, total_profit: -1.42

In [ ]:
def result_to_df(result_list):
    update_time = []
    order_id = []
    order_sub_id = []
    order_side = []
    post_only = []
    create_time = []
    status = []
    open_price = []
    open_home_size = []
    open_foreign_size = []
    close_price = []
    close_home_size = []
    close_foreign_size = []
    order_price = []
    order_home_size = []
    order_foreign_size = []
    profit = []
    fee = []
    total_profit = []
    message = []

    for item in result_list:
        update_time.append(item.update_time)
        order_id.append(item.order_id)
        order_sub_id.append(item.order_sub_id)
        order_side.append(item.order_side)
        post_only.append(item.post_only)
        create_time.append(item.create_time)
        status.append(item.status)
        open_price.append(item.open_price)
        open_home_size.append(item.open_home_size)
        open_foreign_size.append(item.open_foreign_size)
        close_price.append(item.close_price)
        close_home_size.append(item.close_home_size)
        close_foreign_size.append(item.close_foreign_size)
        order_price.append(item.order_price)
        order_home_size.append(item.order_home_size)
        order_foreign_size.append(item.order_foreign_size)
        profit.append(item.profit)
        fee.append(item.fee)
        total_profit.append(item.total_profit)
        message.append(item.message)

    df = pd.DataFrame(
    data={"update_time": update_time, "order_id": order_id, "sub_id": order_sub_id,
          "order_side": order_side, "post_only": post_only, "create_time": create_time,
          "status":  status, 
          "open_price": open_price, "open_home_size": open_home_size, "open_foreign_size": open_foreign_size, 
          "close_price": close_price, "close_home_size": close_home_size, "close_foreign_size": close_foreign_size,
          "order_price": order_price, "order_home_size": order_home_size, "order_foreign_size": order_foreign_size,
          "profit": profit, "fee": fee,
          "total_profit": total_profit, "message": message},
    columns=["update_time", "order_id", "sub_id", "order_side", "post_only",
             "create_time", "status", "open_price", "open_home_size", "open_foreign_size", 
             "close_price", "close_home_size", "close_foreign_size",             
             "order_price", "order_home_size", "order_foreign_size",
             "profit", "fee", "total_profit", "message"])
    df["update_time"] = pd.to_datetime((df["update_time"]), utc=True, unit="us")
    df["create_time"] = pd.to_datetime((df["create_time"]), utc=True, unit="us")
    df["sum_profit"] = df["total_profit"].cumsum()
    # df["sum_pos"] = df["pos_change"].cumsum()
    df = df.set_index("create_time", drop=True)
    
    return df

In [ ]:
df = result_to_df(r)
df

In [ ]:
bn = BinanceMarket("BTCBUSD")
#bn.download(15)


In [ ]:
bn

In [ ]:
rbot.init_debug_log()
bn.download(10)

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
def result_to_df(result_list):
    update_time = []
    order_id = []
    order_sub_id = []
    order_side = []
    post_only = []
    create_time = []
    status = []
    open_price = []
    open_home_size = []
    open_foreign_size = []
    close_price = []
    close_home_size = []
    close_foreign_size = []
    order_price = []
    order_home_size = []
    order_foreign_size = []
    profit = []
    fee = []
    total_profit = []
    message = []

    for item in result_list:
        update_time.append(item.update_time)
        order_id.append(item.order_id)
        order_sub_id.append(item.order_sub_id)
        order_side.append(item.order_side)
        post_only.append(item.post_only)
        create_time.append(item.create_time)
        status.append(item.status)
        open_price.append(item.open_price)
        open_home_size.append(item.open_home_size)
        open_foreign_size.append(item.open_foreign_size)
        close_price.append(item.close_price)
        close_home_size.append(item.close_home_size)
        close_foreign_size.append(item.close_foreign_size)
        order_price.append(item.order_price)
        order_home_size.append(item.home_size)
        order_foreign_size.append(item.order_foreign_size)
        profit.append(item.profit)
        fee.append(item.fee)
        total_profit.append(item.total_profit)
        message.append(item.message)

    df = pd.DataFrame(
    data={"update_time": update_time, "order_id": order_id, "sub_id": order_sub_id,
          "order_side": order_side, "post_only": post_only, "create_time": create_time,
          "status":  status, "open_price": open_price, "open_home_size": open_home_size, 
          "open_foreign_size": open_foreign_size, "close_price": close_price,
          "price": price, "size": size, "volume": volume, "profit": profit, "fee": fee,
          "total_profit": total_profit, "pos_change": position_change, "message": message},
    columns=["update_time", "order_id", "sub_id", "order_type", "post_only",
             "create_time", "status", "open_price", "close_price", "price", "size", "volume",
             "profit", "fee", "total_profit", "pos_change", "message"])
    df["update_time"] = pd.to_datetime((df["update_time"]), utc=True, unit="ms")
    df["create_time"] = pd.to_datetime((df["create_time"]), utc=True, unit="ms")
    df["sum_profit"] = df["total_profit"].cumsum()
    df["sum_pos"] = df["pos_change"].cumsum()
    df = df.set_index("create_time", drop=True)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time